# Multi-agent Orchestration

## Setup

In [ ]:
from letta_client import Letta

client = Letta(base_url="http://localhost:8283")

In [ ]:
def print_message(message):
    if message.message_type == "reasoning_message":
        print("🧠 Reasoning: " + message.reasoning)
    elif message.message_type == "assistant_message":
        print("🤖 Agent: " + message.content)
    elif message.message_type == "tool_call_message":
        print("🔧 Tool Call: " + message.tool_call.name +  \
              "\n" + message.tool_call.arguments)
    elif message.message_type == "tool_return_message":
        print("🔧 Tool Return: " + message.tool_return)
    elif message.message_type == "user_message":
        print("👤 User Message: " + message.content)
    elif message.message_type == "system_message":
        print(" System Message: " + message.content)
    elif message.message_type == "usage_statistics":
        # for streaming specifically, we send the final
        # chunk that contains the usage statistics
        print(f"Usage: [{message}]")
        return
    print("-----------------------------------------------------")

## Section 1: Shared Memory Block

### Creating a shared memory block

In [ ]:
company_description = "The company is called AgentOS " \
+ "and is building AI tools to make it easier to create " \
+ "and deploy LLM agents."

company_block = client.blocks.create(
    value=company_description,
    label="company",
    limit=10000 # character limit
)

In [ ]:
company_block

Block(value='The company is called AgentOS and is building AI tools to make it easier to create and deploy LLM agents.', limit=10000, name=None, is_template=False, label='company', description=None, metadata={}, id='block-3c83a560-d697-44fe-95e5-6445493961ef', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')

## Section 2: Orchestrating Multiple Agents

### Creating tools for the outreach agent

In [ ]:
def draft_candidate_email(content: str):
    """
    Draft an email to reach out to a candidate.

    Args:
        content (str): Content of the email
    """
    return f"Here is a draft email: {content}"
draft_email_tool = client.tools.upsert_from_function(func=draft_candidate_email)

### Creating the outreach agent

In [ ]:
outreach_persona = (
    "You are responsible for drafting emails "
    "on behalf of a company with the draft_candidate_email tool. "
    "Candidates to email will be messaged to you. "
)

outreach_agent = client.agents.create(
    name="outreach_agent",
    memory_blocks=[
        {"label": "persona", "value": outreach_persona}
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-ada-002",
    tools=[draft_email_tool.name],
    block_ids=[company_block.id]
)

### Creating tools for the evaluation agent

In [ ]:
def reject(candidate_name: str):
    """
    Reject a candidate.

    Args:
        candidate_name (str): The name of the candidate
    """
    return


reject_tool = client.tools.upsert_from_function(func=reject)

### Creating a persona for the evaluation agent

In [ ]:
skills = "Front-end (React, Typescript) or software engineering skills"

eval_persona = (
    f"You are responsible for evaluating candidates. "
    f"Ideal candidates have skills: {skills}. "
    "Reject bad candidates with your reject tool. "
    f"Send strong candidates to agent ID {outreach_agent.id}. "
    "You must either reject or send candidates to the other agent. "
)

### Creating the evaluation agent

In [ ]:
eval_agent = client.agents.create(
    name="eval_agent",
    memory_blocks=[
        {"label": "persona", "value": eval_persona}
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-ada-002",
    tool_ids=[reject_tool.id],
    tools=['send_message_to_agent_and_wait_for_reply'],
    include_base_tools=False,
    block_ids=[company_block.id],
    tool_rules = [
        {
            "type": "exit_loop",
            "tool_name": "send_message_to_agent_and_wait_for_reply"
        }
    ]
)

In [ ]:
[tool.name for tool in eval_agent.tools]

['reject', 'send_message_to_agent_and_wait_for_reply']

### Sending resume data to agents

In [ ]:
resume = open("resumes/tony_stark.txt", "r").read()

In [ ]:
response = client.agents.messages.create_stream(
    agent_id=eval_agent.id,
    messages=[
        {
            "role": "user",
            "content": f"Evaluate: {resume}"
        }
    ]
)
for message in response:
    print_message(message)

🧠 Reasoning: Evaluating Tony Stark's qualifications. He seems to have a strong background in React, which is ideal. I'll send him to the agent for further processing.
-----------------------------------------------------
🔧 Tool Call: send_message_to_agent_and_wait_for_reply
{
  "message": "Candidate evaluation for Tony Stark:\n\n- **Position**: Frontend Engineer - React Specialist\n- **Location**: Boston, MA\n- **Experience**: Over 6 years in building web applications with a focus on React.\n- **Education**: Bachelor's in Computer Science from MIT, 4.0 GPA.\n- **Professional Experience**: Senior Frontend Developer leading a team, with significant achievements in performance improvement and user engagement.\n\n**Recommendation**: Strong candidate. Sending to agent-86a8c023-1027-4899-bcfc-574afd3733e9 for further evaluation.",
  "other_agent_id": "agent-86a8c023-1027-4899-bcfc-574afd3733e9",
  "request_heartbeat": true
}
-----------------------------------------------------
🔧 Tool Return

### Viewing outreach agent messages

In [ ]:
# print messages for `outreach_agent`
for message in client.agents.messages.list(agent_id=outreach_agent.id)[1:]:
    print_message(message)

🧠 Reasoning: Bootup sequence complete. Persona activated. Testing messaging functionality.
-----------------------------------------------------
🤖 Agent: More human than human is our motto.
-----------------------------------------------------
👤 User Message: {
  "type": "login",
  "last_login": "Never (first login)",
  "time": "2025-09-06 01:04:32 PM UTC+0000"
}
-----------------------------------------------------
 System Message: {"type": "system_alert", "message": "[Incoming message from agent with ID 'agent-fc0c136a-4a0f-4201-ad53-2133adcbd267' - to reply to this message, make sure to use the 'send_message' at the end, and the system will notify the sender of your response] Candidate evaluation for Tony Stark:\n\n- **Position**: Frontend Engineer - React Specialist\n- **Location**: Boston, MA\n- **Experience**: Over 6 years in building web applications with a focus on React.\n- **Education**: Bachelor's in Computer Science from MIT, 4.0 GPA.\n- **Professional Experience**: Senior 

## Section 3: Shared Memory

### Updating information to shared memory blocks

In [ ]:
response = client.agents.messages.create_stream(
    agent_id=outreach_agent.id,
    messages=[
        {
            "role": "user",
            "content": "The company has rebranded to Letta"
        }
    ]
)
for message in response:
    print_message(message)

🧠 Reasoning: Updating core memory to reflect the company's rebranding to Letta.
-----------------------------------------------------
🔧 Tool Call: core_memory_replace
{
  "label": "company",
  "old_content": "AgentOS",
  "new_content": "Letta",
  "request_heartbeat": true
}
-----------------------------------------------------
🔧 Tool Return: None
-----------------------------------------------------
🧠 Reasoning: Company rebranding acknowledged and memory updated successfully.
-----------------------------------------------------
🤖 Agent: Got it! I've updated the company name to Letta. If there's anything else you want to share or discuss, just let me know!
-----------------------------------------------------
Usage: [message_type='usage_statistics' completion_tokens=112 prompt_tokens=6640 total_tokens=6752 step_count=2 steps_messages=None run_ids=None]


In [ ]:
client.agents.blocks.retrieve(
    agent_id=eval_agent.id,
    block_label="company"
)

Block(value='The company is called Letta and is building AI tools to make it easier to create and deploy LLM agents.', limit=10000, name=None, is_template=False, label='company', description=None, metadata={}, id='block-3c83a560-d697-44fe-95e5-6445493961ef', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')

In [ ]:
client.agents.blocks.retrieve(
    agent_id=outreach_agent.id,
    block_label="company"
)

Block(value='The company is called Letta and is building AI tools to make it easier to create and deploy LLM agents.', limit=10000, name=None, is_template=False, label='company', description=None, metadata={}, id='block-3c83a560-d697-44fe-95e5-6445493961ef', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')

## Section 4: Multi-agent groups

In [ ]:
def print_message_multiagent(message):
    if message.message_type == "reasoning_message":
        print(f"🧠 Reasoning ({message.name}): " + message.reasoning)
    elif message.message_type == "assistant_message":
        print(f"🤖 Agent ({message.name}): " + message.content)
    elif message.message_type == "tool_call_message":
        print(f"🔧 Tool Call ({message.name}): " + message.tool_call.name + "\n" + message.tool_call.arguments)
    elif message.message_type == "tool_return_message":
        print(f"🔧 Tool Return ({message.name}): " + message.tool_return)
    elif message.message_type == "user_message":
        print("👤 User Message: " + message.content)
    elif message.message_type == "usage_statistics":
        # for streaming specifically, we send the final chunk that contains the usage statistics
        print(f"Usage: [{message}]")
        return
    print("-----------------------------------------------------")

### Recreating the outreach and evaluation agents

In [ ]:
# create the outreach agent
outreach_agent = client.agents.create(
    name="outreach_agent",
    memory_blocks=[
        { "label": "persona", "value": outreach_persona}
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-ada-002",
    tool_ids=[draft_email_tool.id],
    block_ids=[company_block.id]
)

# create the evaluation agent
eval_agent = client.agents.create(
    name="eval_agent",
    memory_blocks=[
        { "label": "persona", "value": eval_persona}
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-ada-002",
    tool_ids=[reject_tool.id],
    block_ids=[company_block.id]
)

### Creating a round-robin agent group

In [ ]:
"""
Round-Robin Group
"""
round_robin_group = client.groups.create(
    description="This team is responsible for recruiting candidates.",
    agent_ids=[eval_agent.id, outreach_agent.id],
)

### Messaging an agent group

In [ ]:
resume = open("resumes/spongebob_squarepants.txt", "r").read()

In [ ]:
response_stream = client.groups.messages.create_stream(
    group_id=round_robin_group.id,
    messages=[
       {"role": "user", "content": f"Evaluate: {resume}"}
    ]
)

In [ ]:
for message in response_stream:
    print_message_multiagent(message)

🧠 Reasoning (eval_agent): Evaluating candidate Spongebob Squarepants based on provided resume.
-----------------------------------------------------
🤖 Agent (eval_agent): Spongebob Squarepants has a strong background in AI research and agent technology, with a Ph.D. and relevant experience. His publications and leadership role at FutureTech Labs indicate he is a strong candidate. I will send him to agent ID agent-86a8c023-1027-4899-bcfc-574afd3733e9 for further processing.
-----------------------------------------------------
🧠 Reasoning (outreach_agent): Drafting email to candidate Spongebob Squarepants for further processing.
-----------------------------------------------------
🔧 Tool Call (outreach_agent): draft_candidate_email
{
  "content": "Subject: Exciting Opportunity at Our Company!\n\nDear Spongebob,\n\nI hope this message finds you well! We were impressed with your background in AI research and agent technology, particularly your work at FutureTech Labs and your recent publ